# Progetto HF-VRP-DL

In [14]:
import os
import time
import csv
import math
from gurobipy import Model, GRB, quicksum

def parse_block(lines, start):
    block = []
    i = start
    while i < len(lines):
        l = lines[i].strip()
        if l.startswith('['):
            l = l[1:]
        if l.endswith(']'):
            l = l[:-1]
            if l.strip() == '':
                break
        if l != '':
            block.append(l)
        if lines[i].strip().endswith(']'):
            break
        i += 1
    return block, i

def read_instance(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()

    # Q
    idx_Q = next(i for i, l in enumerate(lines) if l.strip().startswith('Q:'))
    Q_block, _ = parse_block(lines, idx_Q+1)
    Q = [int(x) for x in ' '.join(Q_block).split()]

    # q
    idx_q = next(i for i, l in enumerate(lines) if l.strip().startswith('q:'))
    q_block, _ = parse_block(lines, idx_q+1)
    q = [int(x) for x in ' '.join(q_block).split()]

    # L
    idx_L = next(i for i, l in enumerate(lines) if l.strip().startswith('L:'))
    L_block, _ = parse_block(lines, idx_L+1)
    L = [[int(x) for x in l.split()] for l in L_block]

    # coord
    idx_coord = next(i for i, l in enumerate(lines) if l.strip().startswith('coord:'))
    coord_block, _ = parse_block(lines, idx_coord+1)
    coord = [[int(x) for x in l.split()] for l in coord_block]

    # c
    idx_c = next(i for i, l in enumerate(lines) if l.strip().startswith('c:'))
    c_line = lines[idx_c].split(':')[1].replace('[','').replace(']','').strip()
    c = [float(x) for x in c_line.split()]

    # r
    idx_r = next(i for i, l in enumerate(lines) if l.strip().startswith('r:'))
    r_block, _ = parse_block(lines, idx_r+1)
    r = [[float(x) for x in l.split()] for l in r_block]

    num_ports = len(coord)
    num_ships = len(Q)

    ports = []
    for i in range(num_ports):
        ports.append({
            'id': f'P{i+1}',
            'x': coord[i][0],
            'y': coord[i][1],
            'demand': q[i],
            'draft_limit': min(L[i]) if i < len(L) else 9999
        })

    ships = []
    for k in range(num_ships):
        ships.append({
            'id': f'N{k+1}',
            'capacity': Q[k],
            'cost_per_unit': c[k] if k < len(c) else 1.0,
            'draft_empty': r[0][k] if len(r) > 0 else 4.0,
            'draft_per_unit': 0.02
        })

    # Calcolo delle distanze (t_ij)
    distances = {}
    for i in range(num_ports):
        for j in range(num_ports):
            pi, pj = ports[i], ports[j]
            dist = math.sqrt((pi['x'] - pj['x'])**2 + (pi['y'] - pj['y'])**2)
            distances[(pi['id'], pj['id'])] = round(dist, 2)

    return ships, ports, distances, q, Q, c, r, L

instances_dir = "dataset/HF-VRP-DL-instances"
results_file = "risultati.csv"

with open(results_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['istanza', 'status', 'costo', 'tempo_sec'])

    for filename in sorted(os.listdir(instances_dir)):
        if filename.endswith('.txt'):
            filepath = os.path.join(instances_dir, filename)
            try:
                ships, ports, distances, q, Q, c, r, L = read_instance(filepath)
                port_ids = [p['id'] for p in ports]
                depot = port_ids[0]
                ship_ids = [s['id'] for s in ships]
                num_ports = len(port_ids)
                num_ships = len(ship_ids)
                I0 = port_ids
                I = port_ids[1:]
                S = ship_ids

                demand = {p['id']: p['demand'] for p in ports}
                capacity = {s['id']: s['capacity'] for s in ships}
                cost = {s['id']: s['cost_per_unit'] for s in ships}
                t = distances
                r_is = {}
                for i in port_ids:
                    for k in ship_ids:
                        r_is[(i, k)] = ships[int(k[1:])-1]['draft_empty']  # oppure 0
                L_dict = {}
                for i, p in enumerate(ports):
                    for k in ship_ids:
                        L_dict[(p['id'], k)] = L[i][int(k[1:])-1] if i < len(L) and int(k[1:])-1 < len(L[i]) else 9999

                print(f"File: {filename}")
                print("I (porti senza deposito):", I)
                print("S (navi):", S)
                print("Domanda totale:", sum(demand[i] for i in I))
                print("Capacità totale:", sum(capacity[k] for k in S))
                print("Numero variabili X:", len(I0)*len(I0)*len(S))
                print("Numero variabili Y:", len(I)*len(S))
                print("Numero variabili l:", len(I0)*len(S))
                print("Numero variabili u:", len(I)*len(S))
                print("----")

                m = Model("HF-VRP-DL")
                m.setParam('OutputFlag', 0)

                # Variabili
                X = m.addVars(I0, I0, S, vtype=GRB.BINARY, name="X")
                Y = m.addVars(I, S, vtype=GRB.BINARY, name="Y")
                l = m.addVars(I0, S, vtype=GRB.CONTINUOUS, name="l")  # carico nave s entrando in i
                u = m.addVars(I, S, vtype=GRB.INTEGER, lb=1, ub=len(I), name="u")  # posizione porto i nella sequenza nave s

                # Funzione obiettivo (1)
                m.setObjective(
                    quicksum(cost[k] * t[i, j] * X[i, j, k] for i in I0 for j in I0 for k in S if i != j) +
                    quicksum(r_is[(i, k)] * Y[i, k] for i in I for k in S),
                    GRB.MINIMIZE
                )

                # (2) Ogni porto servito da una sola nave
                for i in I:
                    m.addConstr(quicksum(Y[i, k] for k in S) == 1)

                # (3) Capacità nave
                for k in S:
                    m.addConstr(quicksum(demand[i] * Y[i, k] for i in I) <= capacity[k])

                # (4) Flusso: ogni porto visitato da una nave deve essere raggiunto
                for j in I:
                    for k in S:
                        m.addConstr(quicksum(X[i, j, k] for i in I0 if i != j) == Y[j, k])

                # (5) Flusso: ogni porto visitato da una nave deve essere lasciato
                for i in I:
                    for k in S:
                        m.addConstr(quicksum(X[i, j, k] for j in I0 if j != i) == Y[i, k])

                # (6) Ogni nave entra ed esce dal deposito solo se serve almeno un porto
                for k in S:
                    m.addConstr(quicksum(X[depot, j, k] for j in I) == quicksum(Y[j, k] for j in I))
                    m.addConstr(quicksum(X[j, depot, k] for j in I) == quicksum(Y[j, k] for j in I))

                # (8) Subtour elimination (MTZ-like)
                for i in I:
                    for j in I:
                        if i != j:
                            for k in S:
                                m.addConstr(u[i, k] - u[j, k] + len(I) * X[i, j, k] <= len(I) - 1)

                # (9) Carico nave in ingresso nei porti
                for i in I:
                    for j in I:
                        if i != j:
                            for k in S:
                                m.addConstr(l[j, k] >= l[i, k] - demand[j] * X[i, j, k] - capacity[k] * (1 - X[i, j, k]))

                # (10) Limite di carico massimo per accedere al porto
                for i in I:
                    for k in S:
                        m.addConstr(l[i, k] <= L_dict[(i, k)])

                # (11) Carico nave in uscita dal deposito
                for k in S:
                    m.addConstr(l[depot, k] == quicksum(demand[i] * Y[i, k] for i in I))

                # Dominio variabili
                # X, Y già binarie, u intero, l continuo



                start = time.time()
                m.optimize()
                elapsed = time.time() - start

                if m.status == GRB.OPTIMAL:
                    writer.writerow([filename, "OPTIMAL", round(m.objVal, 2), round(elapsed, 2)])
                else:
                    writer.writerow([filename, "INFEASIBLE", "", round(elapsed, 2)])
            except Exception as e:
                writer.writerow([filename, "ERROR", str(e), ""])

File: r_15_30_30_30_1.txt
I (porti senza deposito): ['P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16']
S (navi): ['N1', 'N2', 'N3']
Domanda totale: 54000
Capacità totale: 180000
Numero variabili X: 768
Numero variabili Y: 45
Numero variabili l: 48
Numero variabili u: 45
----
File: r_15_30_30_30_10.txt
I (porti senza deposito): ['P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16']
S (navi): ['N1', 'N2', 'N3']
Domanda totale: 54000
Capacità totale: 180000
Numero variabili X: 768
Numero variabili Y: 45
Numero variabili l: 48
Numero variabili u: 45
----
File: r_15_30_30_30_2.txt
I (porti senza deposito): ['P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16']
S (navi): ['N1', 'N2', 'N3']
Domanda totale: 54000
Capacità totale: 180000
Numero variabili X: 768
Numero variabili Y: 45
Numero variabili l: 48
Numero variabili u: 45
----
File: r_15_30_30_30_3.txt
I (p

In [16]:
import os
import time
import csv
import math
from gurobipy import Model, GRB, quicksum

def parse_block(lines, start):
    block = []
    i = start
    while i < len(lines):
        l = lines[i].strip()
        if l.startswith('['):
            l = l[1:]
        if l.endswith(']'):
            l = l[:-1]
            if l.strip() == '':
                break
        if l != '':
            block.append(l)
        if lines[i].strip().endswith(']'):
            break
        i += 1
    return block, i

def read_instance(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()

    # Q
    idx_Q = next(i for i, l in enumerate(lines) if l.strip().startswith('Q:'))
    Q_block, _ = parse_block(lines, idx_Q+1)
    Q = [int(x) for x in ' '.join(Q_block).split()]

    # q
    idx_q = next(i for i, l in enumerate(lines) if l.strip().startswith('q:'))
    q_block, _ = parse_block(lines, idx_q+1)
    q = [int(x) for x in ' '.join(q_block).split()]

    # L
    idx_L = next(i for i, l in enumerate(lines) if l.strip().startswith('L:'))
    L_block, _ = parse_block(lines, idx_L+1)
    L = [[int(x) for x in l.split()] for l in L_block]

    # coord
    idx_coord = next(i for i, l in enumerate(lines) if l.strip().startswith('coord:'))
    coord_block, _ = parse_block(lines, idx_coord+1)
    coord = [[int(x) for x in l.split()] for l in coord_block]

    # c
    idx_c = next(i for i, l in enumerate(lines) if l.strip().startswith('c:'))
    c_line = lines[idx_c].split(':')[1].replace('[','').replace(']','').strip()
    c = [float(x) for x in c_line.split()]

    # r
    idx_r = next(i for i, l in enumerate(lines) if l.strip().startswith('r:'))
    r_block, _ = parse_block(lines, idx_r+1)
    r = [[float(x) for x in l.split()] for l in r_block]

    num_ports = len(coord)
    num_ships = len(Q)

    ports = []
    for i in range(num_ports):
        ports.append({
            'id': f'P{i+1}',
            'x': coord[i][0],
            'y': coord[i][1],
            'demand': q[i],
            'draft_limit': min(L[i]) if i < len(L) else 9999
        })

    ships = []
    for k in range(num_ships):
        ships.append({
            'id': f'N{k+1}',
            'capacity': Q[k],
            'cost_per_unit': c[k] if k < len(c) else 1.0,
            'draft_empty': r[0][k] if len(r) > 0 else 4.0,
            'draft_per_unit': 0.02
        })

    # Calcolo delle distanze (t_ij)
    distances = {}
    for i in range(num_ports):
        for j in range(num_ports):
            pi, pj = ports[i], ports[j]
            dist = math.sqrt((pi['x'] - pj['x'])**2 + (pi['y'] - pj['y'])**2)
            distances[(pi['id'], pj['id'])] = round(dist, 2)

    return ships, ports, distances, q, Q, c, r, L

instances_dir = "dataset/HF-VRP-DL-instances"
results_file = "risultati_vi.csv"

with open(results_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['istanza', 'status', 'costo', 'tempo_sec'])

    for filename in sorted(os.listdir(instances_dir)):
        if filename.endswith('.txt'):
            filepath = os.path.join(instances_dir, filename)
            try:
                ships, ports, distances, q, Q, c, r, L = read_instance(filepath)
                port_ids = [p['id'] for p in ports]
                depot = port_ids[0]
                ship_ids = [s['id'] for s in ships]
                num_ports = len(port_ids)
                num_ships = len(ship_ids)
                I0 = port_ids
                I = port_ids[1:]
                S = ship_ids

                demand = {p['id']: p['demand'] for p in ports}
                capacity = {s['id']: s['capacity'] for s in ships}
                cost = {s['id']: s['cost_per_unit'] for s in ships}
                t = distances
                r_is = {}
                for i in port_ids:
                    for k in ship_ids:
                        r_is[(i, k)] = ships[int(k[1:])-1]['draft_empty']
                L_dict = {}
                for i, p in enumerate(ports):
                    for k in ship_ids:
                        L_dict[(p['id'], k)] = L[i][int(k[1:])-1] if i < len(L) and int(k[1:])-1 < len(L[i]) else 9999

                m = Model("HF-VRP-DL")
                m.setParam('OutputFlag', 0)

                # Variabili
                X = m.addVars(I0, I0, S, vtype=GRB.BINARY, name="X")
                Y = m.addVars(I, S, vtype=GRB.BINARY, name="Y")
                l = m.addVars(I0, S, vtype=GRB.CONTINUOUS, name="l")
                u = m.addVars(I, S, vtype=GRB.INTEGER, lb=1, ub=len(I), name="u")

                # Funzione obiettivo (1)
                m.setObjective(
                    quicksum(cost[k] * t[i, j] * X[i, j, k] for i in I0 for j in I0 for k in S if i != j) +
                    quicksum(r_is[(i, k)] * Y[i, k] for i in I for k in S),
                    GRB.MINIMIZE
                )

                # (2) Ogni porto servito da una sola nave
                for i in I:
                    m.addConstr(quicksum(Y[i, k] for k in S) == 1)

                # (3) Capacità nave
                for k in S:
                    m.addConstr(quicksum(demand[i] * Y[i, k] for i in I) <= capacity[k])

                # (4) Flusso: ogni porto visitato da una nave deve essere raggiunto
                for j in I:
                    for k in S:
                        m.addConstr(quicksum(X[i, j, k] for i in I0 if i != j) == Y[j, k])

                # (5) Flusso: ogni porto visitato da una nave deve essere lasciato
                for i in I:
                    for k in S:
                        m.addConstr(quicksum(X[i, j, k] for j in I0 if j != i) == Y[i, k])

                # (6) Ogni nave entra ed esce dal deposito solo se serve almeno un porto
                for k in S:
                    m.addConstr(quicksum(X[depot, j, k] for j in I) == quicksum(Y[j, k] for j in I))
                    m.addConstr(quicksum(X[j, depot, k] for j in I) == quicksum(Y[j, k] for j in I))

                # (8) Subtour elimination (MTZ-like)
                for i in I:
                    for j in I:
                        if i != j:
                            for k in S:
                                m.addConstr(u[i, k] - u[j, k] + len(I) * X[i, j, k] <= len(I) - 1)
                # (9) Carico nave in ingresso nei porti
                for i in I:
                    for j in I:
                        if i != j:
                            for k in S:
                                m.addConstr(l[j, k] >= l[i, k] - demand[j] * X[i, j, k] - capacity[k] * (1 - X[i, j, k]))

                # (10) Limite di carico massimo per accedere al porto
                for i in I:
                    for k in S:
                        m.addConstr(l[i, k] <= L_dict[(i, k)])

                # (11) Carico nave in uscita dal deposito
                for k in S:
                    m.addConstr(l[depot, k] == quicksum(demand[i] * Y[i, k] for i in I))

                # VALID INEQUALITIES
                # V13: elimina assegnazioni impossibili per limiti di pescaggio a vuoto
                for i in I:
                    for k in S:
                        if L_dict[(i, k)] < ships[int(k[1:])-1]['draft_empty']:
                            m.addConstr(Y[i, k] == 0)

                # V14: elimina assegnazioni impossibili per limiti di capacità
                for i in I:
                    for k in S:
                        if demand[i] > capacity[k]:
                            m.addConstr(Y[i, k] == 0)

                start = time.time()
                m.optimize()
                elapsed = time.time() - start

                if m.status == GRB.OPTIMAL:
                    writer.writerow([filename, "OPTIMAL", round(m.objVal, 2), round(elapsed, 2)])
                else:
                    writer.writerow([filename, "INFEASIBLE", "", round(elapsed, 2)])
            except Exception as e:
                writer.writerow([filename, "ERROR", str(e), ""])